In [1]:
from cgi import print_environ
from time import time
import pandas as pd
from sklearn import datasets
from Preprocessor1 import Preprocessor
from sklearn.preprocessing import LabelEncoder
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from ResultStorage import ResultStorage
from tqdm import tqdm
import fasttext

from Vectorizing.CountVectorizer import getCountVectorizer
from Vectorizing.TF_IDF_Vectorizer import getWordLevelVectorizer, getNGramLevelVectorizer
from Embedding.WordEmbedding import createWordEmbedding
from Report import Report
from Report.CrossValidateNN import cross_validate_NN
from lstmModel import create_lstm_model, create_bidirec_lstm_model

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support, make_scorer, accuracy_score, precision_score, recall_score, f1_score

TEXT = "Text"
LABEL = "Label"
CV_splits = 5
sample = True
nb_sample = 5000

2022-03-15 02:25:29.596559: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-15 02:25:29.596584: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def filter_dataframe(df, cat):
	count = 0
	for ind, row in df.iterrows():
		if cat != str(row[LABEL]):
			count += 1
			row[LABEL] = 'Other'
	print(f'{cat} filtered {count} rows in training dataset')

def get_sampling_strategy(df_train, cat):
	sizes = df_train.groupby(LABEL).size()
	indexes = list(sizes.index)
	cat_size = sizes[indexes.index(cat)]
	other_cat_size = int(cat_size/(len(df_train[LABEL].unique())-2))+1
	sampling_stratgy = {}
	for c in df_train[LABEL].unique():
		if c == cat:
			sampling_stratgy[c] = cat_size
		elif c == 'General':
			sampling_stratgy[c] = 0
		else:
			sampling_stratgy[c] = min(other_cat_size, sizes[indexes.index(c)])
	print('Sampling strategy: ', sampling_stratgy)
	return sampling_stratgy

In [4]:
#categories = ["Sequential", "Natural Language Processing", "Audio", "Computer Vision", "Graphs", "Reinforcement Learning", 'Semantic Web']
categories = ['Audio']

In [5]:
datasets = [('readme_train.csv', 'readme_test.csv')]

In [7]:
df_train = pd.read_csv('../data/'+datasets[0][0], sep=';')
df_test = pd.read_csv('../data/'+datasets[0][1], sep = ';')

print(f'df_train: {df_train.shape}')
print(f'df_test: {df_test.shape}')
df_train.drop_duplicates(subset=['Text'], inplace=True, keep=False)
df_test.drop_duplicates(subset=['Text'], inplace=True, keep=False)
print(f'df_train: {df_train.shape}')
print(f'df_test: {df_test.shape}')

df_train: (7703, 4)
df_test: (856, 4)
df_train: (3435, 4)
df_test: (741, 4)


In [8]:
#Preprocessor(df_train).run()
#Preprocessor(df_test).run()
df_train.drop( df_train[ df_train[TEXT] == "" ].index , inplace=True)
df_test.drop( df_test[ df_test[TEXT] == "" ].index , inplace=True)

In [9]:
df_train = df_train.drop(columns = 'Repo')
x_train = df_train[TEXT]
y_train = df_train[LABEL]
x_test = df_test[TEXT]
y_test = df_test[LABEL]

In [10]:
undersample = RandomUnderSampler(sampling_strategy=get_sampling_strategy(df_train, 'Audio'))
x_train, y_train = undersample.fit_resample(x_train.to_frame(TEXT), y_train)

Sampling strategy:  {'Computer Vision': 9, 'Natural Language Processing': 9, 'General': 0, 'Reinforcement Learning': 9, 'Audio': 44, 'Graphs': 9, 'Sequential': 9}


In [11]:
y_train = y_train.to_frame(LABEL)
filter_dataframe(y_train, 'Audio')
y_test = y_test.to_frame(LABEL)
filter_dataframe(y_test, 'Audio')

Audio filtered 45 rows in training dataset
Audio filtered 729 rows in training dataset


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=10000)
tfidf_vect.fit(x_train[TEXT])
xtrain_tfidf = tfidf_vect.transform(x_train[TEXT])
xtest_tfidf = tfidf_vect.transform(x_test)

In [31]:
features = tfidf_vect.get_feature_names()
print(len(features))
print(features)

10
['1', '2', 'audio', 'data', 'dataset', 'example', 'image', 'model', 'set', 'train']


In [45]:
encoder = LabelEncoder()
trainy_count = encoder.fit_transform(y_train)
testy_count = encoder.transform(y_test)
print(trainy_count[0])
print(y_train.head(1))

trainy_tfidf = encoder.fit_transform(y_train)
testy_tfidf = encoder.transform(y_test)

0
   Label
0  Audio


In [42]:
res, model_audio = Report.report(LogisticRegression(max_iter=10000), 'readme_train', 'readme_test', xtrain_tfidf, trainy_tfidf, xtest_tfidf, testy_tfidf, 'Audio', name='LR_TFIDF_RandomUnder', cv=CV_splits, dict_scoring=Report.score_metrics, save=False)

/home/ciuciu/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ciuciu/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ciuciu/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/ciuciu/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/ciuciu/anaconda3/lib/pyt

TypeError: Labels in y_true and y_pred should be of the same type. Got y_true=[0 1] and y_pred=['Audio' 'Other']. Make sure that the predictions provided by the classifier coincides with the true labels.

In [36]:
#text = 'dcgan implement dcgan scratch mnist dataset read dcgan refer origin implement script jyupter notebook faster effici result implement person comput high end gpu servic includ script cpu work edit get start packag tensorflow version 20 kera tensorflow backend import layer numpi mathemat comput pil imag manipul matplotlib plot imag tfutil util kera tensorflow break line 46 gener block origin dcgan input diment vector 128 simplif line 51 gener block 1 channel inform line 66 tensor reshap remov channel blackwhit imag pyplot line 90 batch size divid half one half exampl true half go gener line 109 110 respect gener train get output 1 imag find accuraci batch gener well accuraci discrimin reduc built author nikita kodkani student refer unsupervis represent learn deep convolut gener adversari network'
text = ' audio cat speech cat cat cat image image yolo'
print(model_audio.predict(tfidf_vect.transform([text])))
print(model_audio.predict_proba(tfidf_vect.transform([text])))

[0]
[[0.51263091 0.48736909]]


In [40]:
y_pred = model_audio.predict(xtest_tfidf)

In [41]:
y_unique = [0, 1]
cm = confusion_matrix(testy_tfidf, y_pred, labels=y_unique)
cm_df = pd.DataFrame(cm, index = [y_unique], columns = [y_unique])
print(f"Accuracy: {accuracy_score(testy_tfidf, y_pred)}")
m = precision_recall_fscore_support(testy_tfidf, y_pred, average='weighted')
precision = m[0]
recall = m[1]
f1score = m[2]
print(f"Precision: {m[0]} \nRecall: {m[1]} \nF1-score: {m[2]}")

Accuracy: 0.8758434547908233
Precision: 0.9807028294849715 
Recall: 0.8758434547908233 
F1-score: 0.9204861733325751
